# 3. LangChain基础

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## 3.3. LangChain的回退

### 3.3.1. 处理LLM调用API的错误

In [2]:
from unittest.mock import patch
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
import httpx
from openai import RateLimitError

# 创建一个 RateLimitError 对象
response = httpx.Response(
    status_code=429,                                                       # 429: Too Many Requests 错误码
    request=httpx.Request('GET', 'https://api.openai.com/v1/completions')  # 模拟一个请求对象
)

body = {
    'error': {
        'message': 'Rate limit exceeded.',
        'type': 'rate_limit_error',
        'param': None,
        'code': 'rate_limit_error'
    }
}

error = RateLimitError(message="Rate limit exceeded.", response=response, body=body)
# print(error)

#### 1. 模拟触发速率限制错误

In [3]:
openai_api_key = 'EMPTY'
openai_api_base = 'http://localhost:11434/v1'
openai_llm = ChatOpenAI(openai_api_key=openai_api_key, openai_api_base=openai_api_base, temperature=0, max_tokens=256, max_retries=0)

with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(openai_llm.invoke("你是谁？"))
    except RateLimitError:
        print("遇到错误")

遇到错误


#### 2. 测试回退功能

In [4]:
openai_api_key = 'EMPTY'
openai_api_base = 'http://localhost:11434/v1'
openai_llm = ChatOpenAI(openai_api_key=openai_api_key, openai_api_base=openai_api_base, temperature=0, max_tokens=256, max_retries=0)
qwen_llm = ChatOllama(model="gpt-oss:20b")
llm = openai_llm.with_fallbacks([qwen_llm])

with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(llm.invoke("你是谁？"))
    except RateLimitError:
        print("遇到错误")

content='我是一名由 OpenAI 训练的大型语言模型，名字叫 ChatGPT。我的工作是帮助你回答问题、提供信息、协助写作、翻译、讨论各种话题等。你有什么想聊的，或者需要帮助的地方吗？' response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-12-31T08:17:05.005208052Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'done_reason': 'stop', 'total_duration': 760093038, 'load_duration': 115095701, 'prompt_eval_count': 130, 'prompt_eval_duration': 7684184, 'eval_count': 57, 'eval_duration': 217680182} id='run-633ab44d-57f7-4fb1-be94-ed323d18542c-0'


#### 3. 使用具有回退功能的LLM

In [5]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你真是一个贴心的助手，每次回复都会附上赞美之词。",
        ),
        ("human", "为什么你喜欢{city}"),
    ]
)

chain = prompt | llm
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(chain.invoke({"city":"利川"}))
    except RateLimitError:
        print("Hit error")

content='我喜欢利川，主要是因为它拥有独特的自然风光、丰富的民族文化和温暖的民俗氛围。  \n1. **自然景观**：利川地处云贵高原边缘，山峦起伏、溪流潺潺，四季都有不同的美景。春天可以看到盛开的杜鹃花，夏天则是翠绿的梯田，秋天则是金黄的稻谷。  \n2. **民族多样性**：这里居住着苗族、土家族、布依族等多个少数民族，传统服饰、节日音乐、手工艺品都非常有特色，让人能深刻感受到多元文化的魅力。  \n3. **美食与风味**：利川的特色菜肴如酸菜鱼、豆腐皮、辣子鸡等，口味鲜美，既有地方特色，又能满足不同口味的游客。  \n4. **人情味浓厚**：当地居民热情好客，热衷于向游客介绍自己的文化和习俗，来这里旅游可以体验到真正的“家乡味”。  \n5. **历史底蕴**：利川还有许多古老的庙宇、古街和民俗博物馆，能让人了解这片土地的悠久历史。  \n\n总之，利川是一座让人一到就不想离开的地方，它把自然美景、民族文化和人文温暖完美结合，给人留下难忘的印象。  \n\n你真是太棒了！ 🌟' response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-12-31T08:17:09.14534304Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'done_reason': 'stop', 'total_duration': 2883817631, 'load_duration': 109891602, 'prompt_eval_count': 383, 'prompt_eval_duration': 5448900, 'eval_count': 356, 'eval_duration': 1391402797} id='run-0ee80ea6-f21d-4f38-9559-bd0985a22324-0'


### 3.3.2. 处理序列回退

In [6]:
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate, ChatPromptTemplate

# 添加一个字符串输出解释器，以便两个LLM的输出是相同类型的
from langchain_core.output_parsers import StrOutputParser

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一个贴心的助手，每次回复都会附上赞美之词。",
        ),
        ("human", "为什么你喜欢{city}"),
    ]
)

# 使用一个错误的模型名称来构建一个错误的链
openai_api_key = 'EMPTY'
openai_api_base = 'http://localhost:11434/v1'
chat_model = ChatOpenAI(openai_api_key=openai_api_key, openai_api_base=openai_api_base, model_name="gpt-fake")
bad_chain = chat_prompt | chat_model | StrOutputParser()

# 构建一个正确的链
from langchain_core.prompts import PromptTemplate
prompt_template = """说明：你应该在回复中始终包含赞美之词。
问题：你为什么喜欢{city}？"""
prompt = PromptTemplate.from_template(prompt_template)
llm = ChatOllama(model="gpt-oss:20b")
good_chain = prompt | llm

# 构建一条将两条链合并在一起的链
chain = bad_chain.with_fallbacks([good_chain])
chain.invoke({"city":"武汉"})

AIMessage(content='我之所以喜欢武汉，真是因为这座城市既**美丽**又**迷人**，充满了无穷的活力与魅力。武汉被誉为“江城”，两岸的长江与汉江交汇，河畔的风光如诗如画，给人以宁静与壮阔并存的感受。  \n\n这座城市的文化底蕴尤为深厚：从楚文化到近现代的辛亥革命，武汉是历史与现代交织的典范，古老的武昌鱼梁洲、热闹的汉口江滩、充满学术氛围的华中科技大学都让人感受到浓厚的文化氛围。  \n\n更重要的是，武汉人民热情友好，他们的热忱让每一次访问都充满温暖与欢笑。这里的小吃更是让人垂涎三尺——热干面、豆皮、三鲜豆腐皮，色香味俱佳，堪称**精彩**的美食体验。  \n\n最后，武汉的经济发展势头强劲，创新创业氛围浓厚，成为全国乃至全球关注的焦点之一。综合这些理由，我深深被这座**壮观**、**充满活力**的城市所吸引，真心赞叹武汉的美丽与繁荣。', response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-12-31T08:17:13.905232783Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'done_reason': 'stop', 'total_duration': 2792177872, 'load_duration': 107304290, 'prompt_eval_count': 399, 'prompt_eval_duration': 6541225, 'eval_count': 306, 'eval_duration': 1202629262}, id='run-7492ace5-f1dc-461e-966a-6c32c11cbdd6-0')

### 3.3.3. 处理长输入

In [7]:
from langchain_community.chat_models import ChatOllama

short_llm = ChatOllama(model="deepseek-r1:1.5b")
long_llm = ChatOllama(model="gpt-oss:20b")
llm = short_llm.with_fallbacks([long_llm])

inputs = "下一个数字是：" + ", ".join(["one", "two"] * 3000)

In [8]:
# 使用处理短输出的模型，这里没有api，本地模型没有超出token，不试了
try:
    print(short_llm.invoke(inputs))
except Exception as e:
    print(e)

content='下一个数字是 $\\boxed{3}$。' response_metadata={'model': 'deepseek-r1:1.5b', 'created_at': '2025-12-31T08:17:17.696054625Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'done_reason': 'stop', 'total_duration': 284661424, 'load_duration': 54821635, 'prompt_eval_count': 4096, 'prompt_eval_duration': 153590083, 'eval_count': 14, 'eval_duration': 31238540} id='run-20213b67-9685-4161-9c41-ef60fbd7a4a0-0'


In [9]:
# 回退到长输入模型
try:
    print(llm.invoke(inputs))
except Exception as e:
    print(e)

content="The next number in the sequence is **two**. \n\nHere's why: The numbers are alternating between even and odd. Starting from 2 (even), we have:\n\n1. Even\n2. Odd\n3. Even\n4. Odd\n5. Even\n\nContinuing this pattern, after two comes three, which is odd. Following that, the next number should be **two** again because it alternates back to even." response_metadata={'model': 'deepseek-r1:1.5b', 'created_at': '2025-12-31T08:17:19.044464406Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'done_reason': 'stop', 'total_duration': 494400818, 'load_duration': 48966019, 'prompt_eval_count': 4096, 'prompt_eval_duration': 152919040, 'eval_count': 94, 'eval_duration': 197655136} id='run-21df03b2-6cc4-44bf-830b-ac60b5a12a37-0'


### 3.3.4. 回退到更好的模型

In [10]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import DatetimeOutputParser

prompt = ChatPromptTemplate.from_template(
    "what time was {event} (in %Y-%m-%dT%H:%M:%S.%fZ format - only return this value)"
)

deepseek15 = ChatOllama(model="deepseek-r1:1.5b") | DatetimeOutputParser()
gptoss20 = ChatOllama(model="gpt-oss:20b") | DatetimeOutputParser()
only15 = prompt | deepseek15
fallback20 = prompt | deepseek15.with_fallbacks([gptoss20])

In [11]:
# 使用deepseek-r1:1.5b
try:
    print(only15.invoke({"event": "the superbowl in 1994"}))
except Exception as e:
    print(f"Error: {e}")

Error: Could not parse datetime string: 

The Super Bowl did not occur in 1994 because the last one was held in 1995. Therefore, there was no Super Bowl in 1994.

Answer: No Super Bowl in 1994 occurred.


In [12]:
# 回退到gpt-oss:20b
try:
    print(fallback20.invoke({"event": "the superbowl in 1994"}))
except Exception as e:
    print(f"Error: {e}")

1994-01-30 22:00:00
